In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# from keras.models import Sequential
# from keras.layers import Dense, Dropout
# from keras.callbacks import EarlyStopping
# import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import copy 
import warnings

import os
from torch import optim, nn, utils, Tensor
import lightning.pytorch as pl

In [2]:
# grab data (every game, pc, and day in sales cycle combination: 385570 x 13)
df = pd.read_csv("C:\\Users\\riffere\\Desktop\\pricing_round2.csv")
df = df[df['current_price'] > 0]

In [3]:
df

,Unnamed: 0,event_date,days_out,pc_num,pc_one,original_price,current_price,rolling_30_seats_primary,min_listing_price,max_listing_price,mean_listing_price,rolling_7_seats_logitix,rolling_14_atp_logitix,pct_change_logitix_atp,max_atp_sold,min_atp_sold,max_inventory,remaining_supply
0,0,2022-10-06,80,0,1,184.0,184.0,0.0,0.00,0.00,0.0000,0.0,0.000000,0.000000,43.000000,41.100000,177,47
1,1,2022-10-06,80,1,2,114.0,114.0,0.0,0.00,0.00,0.0000,0.0,0.000000,0.000000,39.200000,34.250000,181,80
2,2,2022-10-06,80,2,3,89.0,89.0,2.0,0.00,0.00,0.0000,0.0,0.000000,0.000000,36.125000,22.510000,210,92
3,3,2022-10-06,80,3,4,79.0,79.0,0.0,0.00,0.00,0.0000,0.0,0.000000,0.000000,23.480000,14.781667,160,97
4,4,2022-10-06,80,4,5,80.0,80.0,2.0,0.00,0.00,0.0000,0.0,0.000000,0.000000,64.350000,20.370000,272,158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228565,228565,2023-04-13,0,25,R,33.0,25.0,224.0,24.00,149.93,59.5574,292.0,16.027278,-0.134623,34.300000,16.362222,1601,58
228566,228566,2023-04-13,0,26,S,33.0,25.0,92.0,24.00,88.44,54.8728,179.0,6.490317,-0.147948,25.534444,21.560000,859,198
228567,228567,2023-04-13,0,27,T,33.0,25.0,3.0,25.00,88.44,40.6900,131.0,6.152800,-0.202536,25.423077,19.800000,1121,61
228568,228568,2023-04-13,0,28,X,319.0,319.0,0.0,337.49,678.90,439.1640,0.0,0.000000,0.000000,0.000000,0.000000,488,-145


In [ ]:
dfx =df[['days_out', 'pc_num', 'original_price', 'rolling_30_seats_primary', 'min_listing_price','max_listing_price','mean_listing_price',
         'rolling_7_seats_logitix','rolling_14_atp_logitix','pct_change_logitix_atp', 'max_atp_sold', 'min_atp_sold', 'max_inventory', 'remaining_supply']]
X = np.array(dfx)
y = np.array(df[['current_price']])